# Analysis

This notebook formats raw data and creates data for map products

In [12]:
import os
import sys
import arcpy
import pandas as pd
import numpy as np
from arcgis.gis import GIS
from arcgis.features import GeoAccessor
from pathlib import Path


In [13]:
# Common paths
dir_project = Path.cwd().parent
dir_data = dir_project/'data'
dir_raw = dir_data/'raw'
dir_processed = dir_data/'processed'
gdb_processed = dir_processed/'processed.gdb'
aprx_name = 'SDR_WQIP_AnnualReport.aprx'

aprx = arcpy.mp.ArcGISProject(dir_project/'arcgis'/aprx_name)



## Appendix C1 Bioassessment

- Data taken from "Maps info spreadsheet.xlsx" recieved 12/23/2021
- Added a tab "All Site Locations" with previous/historic locations
- 12/28/2021 Modified the "2021 Site Locations" tab with the following information: 

|Station|Program|Latitude|Longitude|
|:----|:----|:----|:----|
|901M14155|SMC|33.62139|-117.60683|
|SMC00531|SMC|33.55975|-117.77200|
|ALC04@Aliso Park|Pre-Restoration|33.60117|-117.70517|
|HMPE-1|HMP|33.53714|-117.61085|
|HMPE-2|HMP|33.53841|-117.61071|
|HMPE-3|HMP|33.54332|-117.61186|
|HMPE-4|HMP|33.54543|-117.61123|
|HMPE-5|HMP|33.55239|-117.61089|



#### Function Definitions

In [110]:
# Function definitions
def create_stations(raw_data: dict):
    '''Creates spatial data from spreadsheet coordinates
        
        Parameters
            raw_data: Dictionary of excel sheets from pandas.read_excel
            
        Returns
            spatial data frame
    '''
    
    key = '2021 Site Locations'
    
    df = (
        raw_data[key].loc[0:8]
        .rename(columns=raw_data[key].iloc[0])
        .loc[1:]
        .iloc[:, [1, 2, 3, 4, 5, 6]]
        .rename(columns=lambda col: 'StationDescription' if col == 'Station Description' else col)
    )
    return (pd.DataFrame
            .spatial
            .from_xy(df=df, x_column='Longitude', y_column='Latitude', sr=4326)
           )
    

def create_stations_historic(raw_data: dict):
    '''Creates spatial data for all stations combining current data with historic stations
    
        Parameters
            raw_data: Dictionary of excel sheets from pandas.read_excel
            
        Returns
            pandas data frame
    '''


    key = 'All Site Locations'

    return(
        raw_data[key][['Station', 'Latitude', 'Longitude', 'StationDescription']]
        .append(create_stations(raw_data=raw_data)[['Station', 'Latitude', 'Longitude', 'StationDescription']])
    )


def add_current_scores(raw_data: dict, csci_buckets: list = [0, .62, .79, .92, 1],
                       cram_buckets: list = [0, 43, 62, 81, 100], colors: list =['Red', 'Orange', 'Yellow', 'Blue']):
    '''Add the CSCI and CRAM data to the current monitoring year stations, results are binned
    
        Parameters
            raw_data: Dictionary of excel sheets from pandas.read_excel
            csci_buckets: Bins for CSCI symbology
            cram_buckets: Bins for CRAM symbology
            colors: Text color for bins
            
        Returns
            pandas data frame
    '''
    score_key = '2021 CSCI and CRAM scores'
    
    df = (
        create_stations(raw_data)
        .merge(raw_data[score_key]
               .rename(columns=lambda col: 'Station' if col == 'station' else col)
               .iloc[:, [0, 1, 2]]
               .rename(columns=lambda col: col.replace(' ', ''))
               .rename(columns=lambda col: col.replace('_', '')),
               on=['Station'], how='left')
    )
    
    df['CSCIColor'] = pd.cut(
        x=df['CSCIScore'], bins=csci_buckets, labels=colors)
    df['CRAMColor'] = pd.cut(
        x=df['CRAMScore'], bins=cram_buckets, labels=colors)
    
    return (df
            .assign(** {c: lambda df_, c=c: df_[c].astype('str') for c in ['CRAMColor', 'CSCIColor']})
    )


def add_mean_scores(raw_data: dict, csci_buckets: list = [0, .62, .79, .92, 1],
                    cram_buckets: list = [0, 43, 62, 81, 100], colors: list = ['Red', 'Orange', 'Yellow', 'Blue'],
                    h20_buckets: list =[0, 19, 34, 48, 57]):
    '''Adds the mean historic data and trend data to stations
    
        Parameters
            raw_data: Dictionary of excel sheets from pandas.read_excel
            csci_buckets: Bins for CSCI symbology
            cram_buckets: Bins for CRAM symbology
            colors: Text color for bins
            h20_buckets: Bins for H2O symbology
            
        Returns
            spatial data frame
    '''
    
    correlation_key = 'CRAM CSCI Correlation'
    trend_key = 'OCPW SDR Data 2009-2021'
    
    
    cram = (raw_data[correlation_key]
            .rename(columns=raw_data[correlation_key].iloc[0])
            .loc[1:]
            .iloc[:, [0, 1, 2, 3]]
            .rename(columns=lambda col: col.replace(' ', ''))
            .drop(columns=['Year'])
            .assign(OverallCRAMScore=lambda x: x["OverallCRAMScore"].astype(float))
            .groupby(['Station'])
            .mean()
            )
    
    cram['CRAMColor'] = pd.cut(
        x=cram['OverallCRAMScore'], bins=cram_buckets, labels=colors)
    
    # Fix for 2009 TC-AP station that was in close proximity to SMC00206
    trends = raw_data[trend_key]
    
    trends['Station'] = np.where(
        trends['Station Unique'] == 'TC-AP-2009', 'SMC00206', trends['Station'])
    # End fix

    csci_h20 = (trends
                .iloc[:, [0, 1, 2, 3, 9, 11]]
                .drop(columns=['Cluster Group', 'Year'])
                .groupby(['Station', 'Parameter'])
                .mean()
                .reset_index()
                .pivot(index='Station', columns='Parameter', values='Result')
                )

    csci_h20['CSCIColor'] = pd.cut(
        x=csci_h20['CSCI'], bins=csci_buckets, labels=colors)
    csci_h20['H20Color'] = pd.cut(
        x=csci_h20['H20'], bins=h20_buckets, labels=colors)

    cluster = trends.sort_values(by='Year').groupby(
        'Station')[['Cluster Group', 'Year']].last()

    output = (create_stations_historic(raw_data)
              .merge(cram, on=['Station'], how='left')
              .merge(csci_h20, on=['Station'], how='left')
              .merge(cluster, on=['Station'], how='left')
              .rename(columns=lambda col: col.replace(' ', ''))
              .rename(columns=lambda col: col.replace('_', ''))
              .assign(** {c: lambda df_, c=c: df_[c].astype('str') for c in ['CRAMColor', 'H20Color', 'CSCIColor']}) #python api does not like category dyptes
              )
    return (pd.DataFrame.spatial.from_xy(df=output,
                                         x_column='Longitude',
                                         y_column='Latitude',
                                         sr=4326))

    
    
def format_bioassessment(raw_data: dict, output_gdb: str):
    '''Calls formatting functions and saves to gdb'''
    
    current_year = add_current_scores(raw_data)
    trend_data = add_mean_scores(raw_data)
    
    current_year.spatial.to_featureclass(location=str(gdb_processed/'bio_stations'), overwrite=True)
    trend_data.spatial.to_featureclass(location=str(gdb_processed/'bio_trends'), overwrite=True)
    
    return [current_year, trend_data]


#### Analysis

In [89]:
# Import raw data
excel_data = pd.read_excel(dir_raw/'Maps info spreadsheet.xlsx', sheet_name=None)


In [112]:
# Analyze and save data to geodatabase
results = format_bioassessment(excel_data, gdb_processed)


### Appendix B2 OCFS Maps


#### Function Definitions


In [84]:
def format_ocfs(raw_data: dict, output_gdb: str):
    key = "Data"
    return (
        GeoAccessor.from_xy(raw_data[key], "Long", "Lat").spatial.to_featureclass(
            f'{output_gdb}\\ocfs_stations')
    )


In [85]:
ocfs_data = pd.read_excel(
    dir_raw/'OCFS Coordinates.xlsx', sheet_name=None)

format_ocfs(ocfs_data, gdb_processed)


'h:\\ProjectsH\\Github\\WQIP-Annual-Report-2020-21-Maps\\data\\processed\\processed.gdb\\ocfs_stations'